In [ ]:
# pip install ragas langchain-openai

# RAGAS (Retrieval Augmented Generation Assessment)

## Quick Start

In [ ]:
import os

os.chdir("../")

from ragas.llms import llm_factory
from ragas import evaluate, experiment

# 設定 API Key
from initialization import credential_init
credential_init()

### RAGAS after v0.4 API

LLM領域的API變化的很快，變得不快的框架大概都是快要死掉的。BUCKLE UP~~

In [ ]:
# 2. 準備測試數據 (同前)
from ragas import Dataset

dataset = Dataset(name="test_dataset", backend="local/csv", root_dir=".")
test_data = [
    {"question": "什麼是 RAG?", "answer": "RAG 是檢索增強生成。", "contexts": ["RAG 結合了檢索與生成技術..."], "ground_truth": "RAG 是結合檢索與生成的技術。"},
]

for sample in test_data:
    dataset.append(sample)
dataset.save()

In [ ]:
from openai import AsyncOpenAI
from pydantic import BaseModel
from ragas import experiment
from ragas.metrics.collections import Faithfulness, AnswerRelevancy
from ragas.llms import llm_factory
from ragas.embeddings import embedding_factory

# --- 關鍵部分：定義 LLM ---
# 這裡定義的是「評審模型」，建議使用 GPT-4o 以獲得穩定的評分
client = AsyncOpenAI()
llm = llm_factory("gpt-4o-mini", client=client)
embeddings = embedding_factory("huggingface", "BAAI/bge-m3")

# Define experiment result structure
class ExperimentResult(BaseModel):
    faithfulness: float
    answer_relevancy: float

# Create experiment function
@experiment(ExperimentResult)
async def run_evaluation(row):
    faithfulness = Faithfulness(llm=llm)
    answer_relevancy = AnswerRelevancy(llm=llm, embeddings=embeddings)
    
    faith_result = await faithfulness.ascore(
        response=row.get("answer"),
        retrieved_contexts=row.get("contexts"),
        user_input=row.get("question")
    )

    relevancy_result = await answer_relevancy.ascore(
        user_input=row.get("question"),
        response=row.get("answer")
    )

    return ExperimentResult(
        faithfulness=faith_result.value,
        answer_relevancy=relevancy_result.value
    )

# Run experiment
exp_results = await run_evaluation.arun(dataset)

In [ ]:
exp_results.to_pandas()

# Metrics

## 內容精準度 (Context Precision)內容精準度 
(Context Precision) 是一項衡量指標，用於評估檢索器（Retriever）針對給定查詢時，將檢索內容中相關區塊（Relevant Chunks）的排名優於無關區塊的能力。具體而言，它評估了檢索內容中相關區塊被置於排名頂部的程度。其計算方式為內容中每個區塊的 Precision@k 平均值。Precision@k 是指在排名第 $k$ 位時，相關區塊數量與排名第 $k$ 位總區塊數量的比例。

$$
\text{Context Precision@K} =
\frac{\sum_{k=1}^{K} \left(\text{Precision@k} \times v_k\right)}
{\text{Total number of relevant items in the top } K \text{ results}}
$$


$$
\text{Precision@k} =
\frac{\text{true positives@k}}
{\text{true positives@k} + \text{false positives@k}}
$$

The ContextPrecision metric evaluates whether retrieved contexts are useful for answering a question by comparing each context against a reference answer. Use this when you have a reference answer available.



In [ ]:
from openai import AsyncOpenAI
from ragas.llms import llm_factory
from ragas.metrics.collections import ContextPrecision

# Setup LLM
client = AsyncOpenAI()
llm = llm_factory("gpt-4o-mini", client=client)

# Create metric
scorer = ContextPrecision(llm=llm)

# Evaluate
result = await scorer.ascore(
    user_input="Where is the Eiffel Tower located?",
    reference="The Eiffel Tower is located in Paris.",
    retrieved_contexts=[
        "The Eiffel Tower is located in Paris.",
        "The Brandenburg Gate is located in Berlin."
    ]
)
print(f"Context Precision Score: {result.value}")

In [ ]:
# Evaluate
result = await scorer.ascore(
    user_input="Where is the Eiffel Tower located?",
    reference="The Eiffel Tower is located in Paris.",
    retrieved_contexts=[
        "The Brandenburg Gate is located in Berlin.",
        "The Eiffel Tower is located in Paris."
    ]
)
print(f"Context Precision Score: {result.value}")

In [ ]:
# Evaluate
result = await scorer.ascore(
    user_input="Where is the Eiffel Tower located?",
    reference="The Eiffel Tower is located in Paris.",
    retrieved_contexts=[
        "The weather is great today",
        "The Brandenburg Gate is located in Berlin.",
        "The Eiffel Tower is located in Paris."
    ]
)
print(f"Context Precision Score: {result.value}")

In [ ]:
from ragas.metrics.collections import ContextPrecision


# Define experiment result structure
class ExperimentResult(BaseModel):
    context_precision: float

# Create experiment function
@experiment(ExperimentResult)
async def run_evaluation(row):
    context_precision = ContextPrecision(llm=llm)
    
    context_precision_result = await context_precision.ascore(
        retrieved_contexts=row.get("contexts"),
        user_input=row.get("question"),
        reference=row.get("ground_truth")
    )

    return ExperimentResult(
        context_precision=context_precision_result.value,
    )

dataset = Dataset(name="test_dataset_context_precision", backend="local/csv", root_dir=".")
test_data = [
    {"question": "Where is the Eiffel Tower located?", "answer": "RAG 是檢索增強生成。", 
     "contexts": ["The weather is great today", "The Brandenburg Gate is located in Berlin.", "The Eiffel Tower is located in Paris."], 
     "ground_truth": "RAG 是結合檢索與生成的技術。"},
]

for sample in test_data:
    dataset.append(sample)
dataset.save()

# Run experiment
exp_results = await run_evaluation.arun(dataset, name="context_precision")

In [ ]:
eval_results.to_pandas()

## 回答相關性 (Answer Relevance)
當一個回答能直接且妥善地解決原始問題時，即被視為具有相關性。重要的是，我們對回答相關性的評估並不考慮真實性 (Factuality)，而是針對回答內容不完整或包含冗餘細節的情況進行扣分。

你可以想像成回答是否對於問題有足夠的針對性，可以避免出現雞同鴨講，或是回答是專業的幹話。

為了計算此分數，系統會引導大型語言模型（LLM）針對生成的回答多次產生對應的問題，並測量這些生成問題與原始問題之間的平均餘弦相似度 (Mean Cosine Similarity)。其核心理念在於：如果生成的回答能準確解決初始問題，那麼 LLM 應該能夠從該回答中反向生成與原始問題一致的問題。

$$
\text{Answer Relevancy} =
\frac{1}{N}
\sum_{i=1}^{N}
\text{cosine similarity}(E_{g_i}, E_o)
$$

以下為繁體中文翻譯：

為了計算答案與給定問題之間的相關性，我們採用兩個步驟：

### 步驟一
使用大型語言模型（LLM）從生成的答案中逆向推導出 **n** 個問題變體。  
例如，針對第一個答案，LLM 可能會生成以下可能的問題：

- 問題一：法國位於歐洲的哪個區域？
- 問題二：法國在歐洲中的地理位置是什麼？
- 問題三：你能指出法國位於歐洲的哪個地區嗎？

### 步驟二
計算這些生成問題與實際問題之間的平均餘弦相似度。

### 核心概念
其核心概念在於：如果答案確實正確回應了問題，那麼僅根據該答案，就極有可能重建出原始問題。


In [ ]:
from openai import AsyncOpenAI
from ragas.llms import llm_factory
from ragas.embeddings.base import embedding_factory
from ragas.metrics.collections import AnswerRelevancy

# Setup LLM and embeddings
client = AsyncOpenAI()
llm = llm_factory("gpt-4o-mini", client=client)
embeddings = embedding_factory("huggingface", "BAAI/bge-m3")

# Create metric
scorer = AnswerRelevancy(llm=llm, embeddings=embeddings)

# Evaluate
result = await scorer.ascore(
    user_input="When was the first super bowl?",
    response="The first superbowl was held on Jan 15, 1967"
)
print(f"Answer Relevancy Score: {result.value}")

## 忠實度（Faithfulness）

Faithfulness 指標用於衡量回應內容與所檢索上下文之間在事實上的一致性。其分數範圍為 0 到 1，分數越高代表一致性越好。

若回應中的所有主張都能由檢索到的上下文所支持，則該回應被視為是忠實的（faithful）。

計算方式如下：

1. 辨識回應中的所有主張。
2. 檢查每一項主張是否能從檢索到的上下文中推論得出。
3. 使用以下公式計算忠實度分數：

$$
\text{忠實度分數} = \frac{\text{回應中可由檢索上下文支持的主張數量}}{\text{回應中的主張總數}}
$$

In [ ]:
from openai import AsyncOpenAI
from ragas.llms import llm_factory
from ragas.metrics.collections import Faithfulness

# Setup LLM
client = AsyncOpenAI()
llm = llm_factory("gpt-4o", client=client)

# Create metric
scorer = Faithfulness(llm=llm)

# Evaluate
result = await scorer.ascore(
    user_input="When was the first super bowl?",
    response="The first superbowl was held on Jan 15, 1967",
    retrieved_contexts=[
        "The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles."
    ]
)
print(f"Faithfulness Score: {result.value}")

In [ ]:
# Evaluate
result = await scorer.ascore(
    user_input="Where and when was Einstein born?",
    response="Einstein was born in Germany, Einstein was born on 1st March 1879.",
    retrieved_contexts=[
        "Albert Einstein (born 14 March 1879) was a German-born theoretical physicist, widely held to be one of the greatest and most influential scientists of all time"
    ]
)
print(f"Faithfulness Score: {result.value}")

## 噪音敏感度（Noise Sensitivity）

### 🔍 理解 RAGAS 中的噪音敏感度

噪音敏感度用於衡量生成模型（LLM）在面對檢索到的上下文中**不正確或不相關資訊**時，受到干擾的程度。然而，從原始碼的實作可以看出，這個指標是**具條件性的**，需要謹慎解讀。

---

### 1. 邏輯運算（懲罰計算）

根據 RAGAS 的實作方式，答案中的某一項主張，只有在**同時滿足三個布林條件**時，才會觸發「噪音懲罰（Noise Penalty）」。

對於一個主張 $c$，其懲罰可表示為：

$$
\text{Penalty}(c) = (\text{Context} \cap \text{GT}) \wedge (\text{Context} \cap \text{Answer}) \wedge \neg \text{GT}
$$

其中：

- $(\text{Context} \cap \text{GT})$：上下文必須是**相關的**（包含正確答案／事實）。
- $(\text{Context} \cap \text{Answer})$：模型必須對該上下文保持**忠實**（答案確實是從上下文中抽取的）。
- $\neg \text{GT}$：該主張必須是**錯誤的**（不存在於 Ground Truth 中）。

---

### 2. 「零分」的細微差異（Zero Score Nuance）

分數為 **0** 並不一定代表系統表現完美。由於使用的是 $\wedge$（AND）邏輯，只要鏈條中的**任何一個條件不成立**，最終分數就會變成 0。

| 如果…… | 結果 | 原因 |
| :--- | :--- | :--- |
| **檢索品質不佳** | 分數 = 0 | 若上下文不包含 Ground Truth，「相關性」條件為 False |
| **模型產生幻覺** | 分數 = 0 | 若答案不在上下文中，「忠實性」條件為 False |
| **模型表現完美** | 分數 = 0 | 若答案符合 Ground Truth，「錯誤性」條件為 False |

> **⚠️ 注意事項：**  
> 請務必將噪音敏感度與 **Context Recall（上下文召回率）** 及 **Faithfulness（忠實度）** 一併評估，以避免將「0 分」誤判為正向結果。


In [ ]:
from ragas.metrics.collections import NoiseSensitivity

# Setup LLM
client = AsyncOpenAI()
llm = llm_factory("gpt-4o-mini", client=client)

# Create metric
scorer = NoiseSensitivity(llm=llm)

In [ ]:
result = await scorer.ascore(
        user_input="What is the Life Insurance Corporation of India (LIC) known for?",
        response="The Life Insurance Corporation of India (LIC) is the largest insurance company in India, known for its vast portfolio of investments. LIC contributes to the financial stability of the country.",
        reference="The Life Insurance Corporation of India (LIC) is the largest insurance company in India, established in 1956 through the nationalization of the insurance industry. It is known for managing a large portfolio of investments.",
        retrieved_contexts=[
            "The Life Insurance Corporation of India (LIC) was established in 1956 following the nationalization of the insurance industry in India.",
            "LIC is the largest insurance company in India, with a vast network of policyholders and huge investments.",
            "As the largest institutional investor in India, LIC manages substantial funds, contributing to the financial stability of the country.",
            "The Indian economy is one of the fastest-growing major economies in the world, thanks to sectors like finance, technology, manufacturing etc."
        ]
    )
    print(f"Noise Sensitivity Score: {result.value}")

## 答案準確度（Answer Accuracy）

答案準確度用於衡量模型回應與給定問題之**參考標準答案（Ground Truth）**之間的一致程度。  
此指標透過兩個獨立的「**LLM-as-a-Judge**」提示進行評估，每個提示都會回傳一個評分（0、2 或 4）。

接著，該指標會將評分轉換為 \([0, 1]\) 的區間，並對兩位評審的分數取平均值。  
分數越高，表示模型的回答越接近參考答案。

評分對應意義如下：

- **0** → 回應不正確，或未回答與參考答案相同的問題。
- **2** → 回應與參考答案**部分一致**。
- **4** → 回應與參考答案**完全一致**。

In [ ]:
from openai import AsyncOpenAI
from ragas.llms import llm_factory
from ragas.metrics.collections import AnswerAccuracy

# Setup LLM
client = AsyncOpenAI()
llm = llm_factory("gpt-4o-mini", client=client)

# Create metric
scorer = AnswerAccuracy(llm=llm)

# Evaluate
result = await scorer.ascore(
    user_input="When was Einstein born?",
    response="Albert Einstein was born in 1879.",
    reference="Albert Einstein was born in 1879."
)
print(f"Answer Accuracy Score: {result.value}")

## Testset Generation for RAG

### Overview
In this tutorial, we'll explore the test set generation module in Ragas to create a synthetic test set for a Retrieval-Augmented Generation (RAG)-based question-answering bot. Our goal is to design a Ragas Airline Assistant capable of answering customer queries on various topics, including:

- Flight booking
- Flight changes and cancellations
- Baggage policies
- Viewing reservations
- Flight delays
- In-flight services
- Special assistance
 
To make sure our synthetic dataset is as realistic and diverse as possible, we will create different customer personas. Each persona will represent distinct traveler types and behaviors, helping us build a comprehensive and representative test set. This approach ensures that we can thoroughly evaluate the effectiveness and robustness of our RAG model.

Source: https://docs.ragas.io/en/v0.4.0/howtos/applications/singlehop_testset_gen/

In [ ]:
!git clone https://huggingface.co/datasets/vibrantlabsai/ragas-airline-dataset

In [ ]:
# pip install unstructured

In [ ]:
# !pip install unstructured[md]

In [ ]:
from langchain_community.document_loaders import DirectoryLoader

path = "ragas-airline-dataset"
loader = DirectoryLoader(path, glob="**/*.md")
docs = loader.load()

In [ ]:
import os

os.chdir("../")

from initialization import credential_init

credential_init()



In [ ]:
from openai import AsyncOpenAI
from ragas.llms import llm_factory
from ragas.embeddings import embedding_factory

client = AsyncOpenAI()
generator_llm = llm_factory("gpt-4o-mini", client=client)
embeddings = embedding_factory("huggingface", "BAAI/bge-m3")

## Create Knowledge Graph
Create a base knowledge graph with the documents

ToDo: use the earily version of RAGAS to explain knowledge graph

In [ ]:
from ragas.testset.graph import KnowledgeGraph
from ragas.testset.graph import Node, NodeType


kg = KnowledgeGraph()

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )

kg

## Setup the transforms

In this tutorial, we create a Single Hop Query dataset using a knowledge graph built solely from nodes. To enhance our graph and improve query generation, we apply three key transformations:

- **Headline Extraction**: Uses a language model to extract clear section titles from each document (e.g., “Airline Initiated Cancellations” from flight cancellations.md). These titles isolate specific topics and provide direct context for generating focused questions.
- **Headline Splitting**: Divides documents into manageable subsections based on the extracted headlines. This increases the number of nodes and ensures more granular, context-specific query generation.
- **Keyphrase Extraction**: Identifies core thematic keyphrases (such as key seating information) that serve as semantic seed points, enriching the diversity and relevance of the generated queries.

In [ ]:
from ragas.testset.transforms import apply_transforms
from ragas.testset.transforms import HeadlinesExtractor, HeadlineSplitter, KeyphrasesExtractor

headline_extractor = HeadlinesExtractor(llm=generator_llm, max_num=20)
headline_splitter = HeadlineSplitter(max_tokens=1500)
keyphrase_extractor = KeyphrasesExtractor(llm=generator_llm)

transforms = [
    headline_extractor,
    headline_splitter,
    keyphrase_extractor
]

apply_transforms(kg, transforms=transforms)

In [ ]:
HeadlinesExtractor?

## Configuring Personas for Query Generation
Personas provide context and perspective, ensuring that generated queries are natural, user-specific, and diverse. By tailoring queries to different user viewpoints, our test set covers a wide range of scenarios:

- **First Time Flier**: Generates queries with detailed, step-by-step guidance, catering to newcomers who need clear instructions.
- **Frequent Flier**: Produces concise, efficiency-focused queries for experienced travelers.
- **Angry Business Class Flier**: Yields queries with a critical, urgent tone to reflect high expectations and immediate resolution demands.

In [ ]:
from ragas.testset.persona import Persona

persona_first_time_flier = Persona(
    name="First Time Flier",
    role_description="Is flying for the first time and may feel anxious. Needs clear guidance on flight procedures, safety protocols, and what to expect throughout the journey.",
)

persona_frequent_flier = Persona(
    name="Frequent Flier",
    role_description="Travels regularly and values efficiency and comfort. Interested in loyalty programs, express services, and a seamless travel experience.",
)

persona_angry_business_flier = Persona(
    name="Angry Business Class Flier",
    role_description="Demands top-tier service and is easily irritated by any delays or issues. Expects immediate resolutions and is quick to express frustration if standards are not met.",
)

personas = [persona_first_time_flier, persona_frequent_flier, persona_angry_business_flier]

In [ ]:
from ragas.testset.synthesizers.single_hop.specific import SingleHopSpecificQuerySynthesizer

query_distibution = [
    (
        SingleHopSpecificQuerySynthesizer(llm=generator_llm, property_name="headlines"),
        0.5,
    ),
    (
        SingleHopSpecificQuerySynthesizer(
            llm=generator_llm, property_name="keyphrases"
        ),
        0.5,
    ),
]

In [ ]:
SingleHopSpecificQuerySynthesizer?

In [ ]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(
    llm=generator_llm,
    embedding_model=embeddings,
    knowledge_graph=kg,
    persona_list=personas,
)

In [ ]:
TestsetGenerator?

In [ ]:
testset = generator.generate(testset_size=10, query_distribution=query_distibution)
testset.to_pandas()